In [1]:
import numpy as np
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import pickle
from stochastic_cramer_rao import cramer_rao

# Number of sensors
M = 12

# Number of nodes
K = 6

# Locations of the nodes with respect to initial node
locations = (np.array([0,0]),np.array([0.45,0.99]),np.array([3.02,0.45]),np.array([5.61,0.90])
             ,np.array([8.03,1.46]),np.array([8.70,0.50]))

# Direction of arrivals
angles = np.array([5,-14,-10])*2*(np.pi)/360
K_l = np.array([np.sin(angles), np.cos(angles)])

# Steering vector is implemented. It should have the shape: (12,3)
for j in range(0,len(locations)):
    for i in range(0,len(angles)):
        first_part = np.exp(1j*np.pi*locations[j].dot(K_l[:,i]))
        second_part = np.array([1, np.exp(1j*np.pi*1*np.sin(angles[i]))])
        A_k = np.transpose(np.array(first_part*second_part, ndmin=2))
        if i == 0:
            B_k = A_k
        else:
            B_k = np.concatenate((B_k, A_k), axis=1)
        
        if i == len(angles)-1:
            if j == 0:
                A = np.transpose(B_k)
            else:
                A = np.concatenate((A, np.transpose(B_k)), axis=1)

A = np.transpose(A)

# uncomment here and comment for N_samples if you want to iterate over SNR
for x in range(7,8):
    #MSE = np.zeros(1000)
    #MSE = np.zeros(80)
    #cramer = np.zeros(1000)
    cramer = np.zeros(80)
    for snr_dB in range(-20,60):
    #for N_samples in range(2,1002):
        
        for i in range(500):

            #snr_dB = 10
            # Signal(A*s) to noise(n) ratio
            received_snr = 10**(snr_dB/20)
            ratio_As_to_s = 1/3
            snr = received_snr*ratio_As_to_s
            N_samples = 100
            #snr = 10**(snr_dB/20)

            # Source signal implementation (shape: (3,500))
            signal = np.random.normal(0,np.sqrt(snr),(3,N_samples))
            #w = np.atleast_2d([np.pi/3, np.pi/4, np.pi/5]).T
            #signal = (np.sqrt(snr/3))*np.exp(1j*w*(np.atleast_2d(np.arange(1,N_samples+1))))

            # Received signal power on sensors
            signal_power = sum(sum(np.abs(A.dot(signal))**2))

            # Noise signal implementation (shape: (12,500))
            noise = np.random.normal(0,np.sqrt(0.5),(12,N_samples)) + 1j*np.random.normal(0,np.sqrt(0.5),(12,N_samples))
            noise_power  = sum(sum(np.abs(noise)**2))
            if i == 0:
                print("SIGNAL TO NOISE RATIO")
                print(signal_power/noise_power)

            # Received signal (shape: (12,500))
            z = A.dot(signal) + noise

            # Sample covariance matrix
            R_sample = z.dot(z.conj().T)

            # Eigenvalue and eigenvectors
            w_sample, v_sample = np.linalg.eig(R_sample)
            if i == 0 and snr_dB == -20:
                print()
                print("EIGENVALUES OF SAMPLE COVARIANCE MATRIX")
                print(w_sample[0])
                print(w_sample[1])
                print(w_sample[2])
                print(w_sample[3])

            # Sensor Selection Matrix (shape: (12,6))
            T = np.array([[1,0,0,0,0,0],
                          [1,0,0,0,0,0],
                          [0,1,0,0,0,0],
                          [0,1,0,0,0,0],
                          [0,0,1,0,0,0],
                          [0,0,1,0,0,0],
                          [0,0,0,1,0,0],
                          [0,0,0,1,0,0],
                          [0,0,0,0,1,0],
                          [0,0,0,0,1,0],
                          [0,0,0,0,0,1],
                          [0,0,0,0,0,1]])

            # Push-Sum Matrix (shape: (6,6))
            P_push = np.array([[0.2,0.2,0.2,0  ,0  ,0],
                          [0.2,0.2,0.2,0  ,0  ,0],
                          [0.6,0.6,0.2,0.2,0  ,0],
                          [0  ,0  ,0.4,0.2,0.2,0.2],
                          [0  ,0  ,0  ,0.2,0.2,0.2],
                          [0  ,0  ,0  ,0.4,0.6,0.6]])

            # Average-Consensus Matrix (shape: (6,6))
            P_ave = np.array([[0.5,0.25,0.25,0  ,0  ,0],
                          [0.25,0.5,0.25,0  ,0  ,0],
                          [0.25,0.25,0.25,0.25,0  ,0],
                          [0  ,0  ,0.25,0.25,0.25,0.25],
                          [0  ,0  ,0  ,0.25,0.5,0.25],
                          [0  ,0  ,0  ,0.25,0.25,0.5]])

            # Weight Vector  (shape: (6,1))
            w = np.atleast_2d([1,1,1,1,1,1]).T

            if x == 0:
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,10)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,10)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
                R = R_push
            if x == 1:
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,20)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,20)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))
                R = R_push
            if x == 2:
                # Push-Sum Covariance Matrix Estimation
                R_push_numerator = np.multiply(T.dot(np.linalg.matrix_power(P_push,30)).dot(T.T), R_sample)
                R_push_denominator = T.dot(np.linalg.matrix_power(P_push,30)).dot(w).dot(np.ones((1,6))).dot(T.T)

                # Push Sum Covariance Matrix (shape: (12,12))
                R_push = K*np.multiply(R_push_numerator, (1/(R_push_denominator)))  
                R = R_push

            if x == 3:
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,10)).dot(T.T), R_sample)
                R = R_ave_con

            if x == 4:
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,20)).dot(T.T), R_sample)
                R = R_ave_con         

            if x == 5:
                # Average Consensus Covariance Matrix Estimation      
                R_ave_con = K * np.multiply(T.dot(np.linalg.matrix_power(P_ave,30)).dot(T.T), R_sample)
                R = R_ave_con

            if x == 6 or x == 7 or x == 8  :
                # Conventional ESPRIT Algorithm      
                R = R_sample               

    #         if i == 0 and snr_dB == -20:
    #             print()
    #             print("                            R_push")
    #             print("****************************************************************")
    #             print("****************************************************************")
    #             print(R_push)
    #             print()
    #             print("                           R_sample")
    #             print("****************************************************************")
    #             print("****************************************************************")
    #             print(R_sample)
    #             print()
    #             # Check Convergence Status
    #             print(R_push == R_sample)

            w_push, v_push = np.linalg.eig(R)

            # Upper group selection matrix J_up
            J_up = np.kron(np.eye(6),np.array([1,0]))

            # Lower group selection matrix J_down
            J_down = np.kron(np.eye(6),np.array([0,1]))

            # Push-Sum estimated signal eigenvector matrices
            U_s_push = v_push[:,:3]

            # Upper signal eigenvectors
            U_s_up = J_up.dot(U_s_push)

            # Lower signal eigenvectors
            U_s_down = J_down.dot(U_s_push)

            # Matrix including knowledge about DOAs of the source signals
            psi = np.linalg.inv((U_s_up.conj().T).dot(U_s_up)).dot((U_s_up.conj().T)).dot(U_s_down)

            w2, v2 = np.linalg.eig(psi)
            doa = []
            doa.append(np.arcsin(np.angle(w2[0])/np.pi)/(2*np.pi)*360)
            doa.append(np.arcsin(np.angle(w2[1])/np.pi)/(2*np.pi)*360)
            doa.append(np.arcsin(np.angle(w2[2])/np.pi)/(2*np.pi)*360)

            if i == 0:
                print()
                print("  DOAs of the source signals in degrees with SNR: " + str(snr_dB) )
                print("  DOAs of the source signals in degrees with N_samples: " + str(N_samples) )
                print("****************************************************************")
                print("****************************************************************")
                print("DOA of the first source signal:   " + str(doa[0]))
                print("DOA of the second source signal:   " + str(doa[1]))
                print("DOA of the third source signal:   " + str(doa[2]))

            # Derivation matrix
            D = np.vstack((np.gradient(A[:,0]), np.gradient(A[:,1]), np.gradient(A[:,2]))).T
            # Orthogonal complement of the projection matrix
            P_A = np.identity(12) - A.dot(np.linalg.inv(A.conj().T.dot(A))).dot(A.conj().T)
            # Emitter covariance matrix
            S = signal.dot(signal.conj().T)

            #print(np.real(D.conj().T.dot(P_A).dot(D)).shape)
            #print((S.dot(A.conj().T)))
            #print((S.dot(A.conj().T).dot(np.linalg.inv(R_sample))))
            #print((S.dot(A.conj().T).dot(np.linalg.inv(R_sample)).dot(A)))
            #print(((S.dot(A.conj().T).dot(np.linalg.inv(R_sample)).dot(A).dot(S)).T).shape)
            diff_1 = min(abs(doa[0]-angles*360/(2*np.pi)))
            diff_2 = min(abs(doa[1]-angles*360/(2*np.pi)))
            diff_3 = min(abs(doa[2]-angles*360/(2*np.pi)))

            # Uncomment here if you want to iterate over SNR
            # Here in every iteration doa angles change and we take 1/500 times of it. This corresponds to Monte-Carlo simulation.
            
            # MSE[snr_dB+20] = MSE[snr_dB+20] + 1/3*1/500*((diff_1)**2+(diff_2)**2+(diff_3)**2)
            # MSE[N_samples-2] = MSE[N_samples-2] + 1/3*1/500*((diff_1)**2+(diff_2)**2+(diff_3)**2)
            # cramer[N_samples-2] = cramer[N_samples-2] + 1/500*cramer_rao(A, signal, angles, locations)
            cramer[snr_dB+20] = cramer[snr_dB+20] + 1/500*cramer_rao(A, signal, angles, locations)
            
            if i == 499: 
                #print("MSE")
                #print(MSE[snr_dB+20])
                #print(MSE[N_samples-2])
                print("Cramer Rao Bound")
                #print(cramer[N_samples-2])
                print(cramer[snr_dB+20])
    
    RMSE = np.sqrt(MSE)
    # load the results in memory since it takes too much time to obtain it again.(1000 iteration for N=1000)
    # (Don't forget to change the name when you change some parameters above)
    if x == 0:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_10.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 1:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_20.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)    
    if x == 2:
        with open("Push_Sum_ESPRIT_SNR_10dB_P_30.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 3:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_10.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 4:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_20.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 5:
        with open("Average_Consensus_ESPRIT_SNR_10dB_P_30.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 6:
        with open("Conventional_ESPRIT_SNR_10dB.txt", "wb") as fp:   #Pickling
            pickle.dump(RMSE, fp)
    if x == 7:
        with open("Cramer_Rao_Bound_1000_Samples.txt", "wb") as fp:   #Pickling
            pickle.dump(cramer, fp)
    if x == 8:
        with open("Cramer_Rao_Bound_SNR_10dB.txt", "wb") as fp:   #Pickling
            pickle.dump(cramer, fp) 


SIGNAL TO NOISE RATIO
0.09977694449733975

EIGENVALUES OF SAMPLE COVARIANCE MATRIX
(201.1130537517245-3.4866012381319325e-15j)
(160.59203436975432+9.596241874596758e-15j)
(147.95456781158197+1.4186550685653882e-15j)
(137.1767097304799-9.975647227733562e-15j)

  DOAs of the source signals in degrees with SNR: -20
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   -12.212839502646839
DOA of the second source signal:   -18.694663538273804
DOA of the third source signal:   52.13626385250658
Cramer Rao Bound
1.77388960019405
SIGNAL TO NOISE RATIO
0.11068709154044395

  DOAs of the source signals in degrees with SNR: -19
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the fir

Cramer Rao Bound
0.6907277902216505
SIGNAL TO NOISE RATIO
0.7910606235694548

  DOAs of the source signals in degrees with SNR: -3
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   -16.665637265482147
DOA of the second source signal:   -6.998154310117591
DOA of the third source signal:   6.2469416193355976
Cramer Rao Bound
0.6517435210306392
SIGNAL TO NOISE RATIO
0.6915719685740976

  DOAs of the source signals in degrees with SNR: -2
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   9.733673269820155
DOA of the second source signal:   -12.831038187978667
DOA of the third source signal:   -11.81733263355007
Cramer Rao Bound
0.6160188966672495
S

Cramer Rao Bound
0.24357511362183235
SIGNAL TO NOISE RATIO
5.354451241727818

  DOAs of the source signals in degrees with SNR: 15
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.10854977403053
DOA of the second source signal:   -9.435528740666035
DOA of the third source signal:   -14.329755765903677
Cramer Rao Bound
0.23015995863796357
SIGNAL TO NOISE RATIO
6.310537806586481

  DOAs of the source signals in degrees with SNR: 16
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.860896838264034
DOA of the second source signal:   -14.681482820875148
DOA of the third source signal:   -7.8459653828402836
Cramer Rao Bound
0.21741024944106274
S

Cramer Rao Bound
0.0866884205666385
SIGNAL TO NOISE RATIO
48.38865622852

  DOAs of the source signals in degrees with SNR: 33
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.787387140376588
DOA of the second source signal:   -10.081805137865494
DOA of the third source signal:   -14.450201708456028
Cramer Rao Bound
0.08169845658057584
SIGNAL TO NOISE RATIO
43.06448130073098

  DOAs of the source signals in degrees with SNR: 34
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.887312352688005
DOA of the second source signal:   -13.67354444119207
DOA of the third source signal:   -10.186427519690698
Cramer Rao Bound
0.07739646331582939
SIGN

Cramer Rao Bound
0.030694602021865035
SIGNAL TO NOISE RATIO
343.39840517182716

  DOAs of the source signals in degrees with SNR: 51
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   4.935834688280634
DOA of the second source signal:   -10.136457780815718
DOA of the third source signal:   -14.115696843643278
Cramer Rao Bound
0.028980633090211698
SIGNAL TO NOISE RATIO
411.3400498075004

  DOAs of the source signals in degrees with SNR: 52
  DOAs of the source signals in degrees with N_samples: 100
****************************************************************
****************************************************************
DOA of the first source signal:   5.0106971114795
DOA of the second source signal:   -14.025971835893325
DOA of the third source signal:   -9.80976621251876
Cramer Rao Bound
0.027441230404266475

NameError: name 'MSE' is not defined

In [2]:
with open("Cramer_Rao_Bound_SNR_10dB.txt", "wb") as fp:   #Pickling
    pickle.dump(cramer, fp) 